## Pre-processing for vesicle images

In [16]:
import numpy as np
from skimage import data, filters
import h5py
import napari
import os
import scipy
import pandas as pd

import cupy as cp
from cupyx.scipy import ndimage as ndi
import cv2
from scipy.spatial import cKDTree, distance

In [192]:
def peak_det(img, testmode=True, min_dist=20, thresh_abs=0.12, num_peaks=100, noise_level=250, smoothing_radius=2, border_limit=10):
    "Peak_local_max all-in-one as a combo of opencv and cupy"
    img = cp.array(img)
    img[img<0] = 0
    thresh_abs = thresh_abs
    size = int(2 * min_dist + 1)
    img_ana = img.astype('uint16')
    # get filter structuring element
    footprint = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=[size,size])
    # maximum filter (dilation + equal)
    image_max = cv2.dilate(img_ana.get(), kernel=footprint)
    img_rat_dil = np.copy(image_max)
    
    #return image, image_max
    mask = cp.equal(img_ana,cp.array(image_max))
    img_peak_mask = np.copy(mask)
    mask &= cp.greater(img_ana, thresh_abs)
    img_peak_mask_filt = np.copy(mask)
    
    # get coordinates of peaks
    coordinates = cp.nonzero(mask)
    intensities = img_ana[coordinates]
    # highest peak first
    idx_maxsort = cp.argsort(-intensities).get()
    coordinates = tuple(arr.get() for arr in coordinates)
    coordinates = np.transpose(coordinates)[idx_maxsort]

    coordinates = cp.array(coordinates)
    coordinates = cp.fliplr(coordinates)
    coordinates = coordinates.get()

    # remove everything on the border (takes ~2-3ms if there are a lot of detected coordinates, but usually this is not the case)
    imsize = cp.shape(img)[0]
    idxremove = []
    for idx, coordpair in enumerate(coordinates):
        if coordpair[0] < border_limit or coordpair[0] > imsize - border_limit or coordpair[1] < border_limit or coordpair[1] > imsize - border_limit:
            idxremove.append(idx)
    coordinates = np.delete(coordinates,idxremove,axis=0)

    # remove everyhting down to a certain length
    if len(coordinates) > num_peaks:
        coordinates = coordinates[:int(num_peaks),:]

    if testmode:
        img_ana = img_ana.get()
        return coordinates, img_ana, img_rat_dil, img_peak_mask, img_peak_mask_filt
    else:
        return coordinates
    
def dist(a,b):
    return np.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2)

In [4]:
folder = 'E:\\PhD\\ImSwitchData\\2022-01-18\\sample5'
filelist = os.listdir(folder)
print(filelist)
filenum = 0

['14h34m01s619390us_rec_etSTEDCamera.hdf5', '14h34m55s341975us_rec_etSTEDCamera.hdf5', '14h35m28s068683us_rec_etSTEDCamera.hdf5', '14h36m09s073003us_rec_etSTEDCamera.hdf5', '14h42m32s078793us_rec_etSTEDCamera.hdf5', '14h44m21s781974us_rec_etSTEDCamera.hdf5', '14h44m47s002972us_rec_etSTEDCamera.hdf5', '14h52m02s440972us_rec_etSTEDCamera.hdf5', '15h01m10s293185us_rec_etSTEDCamera.hdf5', '15h02m15s246991us_rec_etSTEDCamera.hdf5', '15h02m56s645009us_rec_etSTEDCamera.hdf5', '15h03m17s004581us_rec_etSTEDCamera.hdf5']


In [336]:
file = h5py.File(os.path.join(folder, filelist[filenum]), 'r')
imgstack_raw = file['data'][:]
imgstack_sm = scipy.ndimage.gaussian_filter(imgstack_raw, sigma=0.6)
imgstack_sm = imgstack_sm[110:130]

In [ ]:
viewer = napari.Viewer()

# raw img
viewer.add_image(imgstack_sm, name='raw', colormap='inferno', visible=True)

# mean img
#img_mean = np.mean(imgstack_sm,axis=0)
#print(np.shape(img_mean))
#viewer.add_image(img_mean, name='mean', colormap='inferno', visible=True)

# img - mean img
#img_meansub = imgstack_sm - img_mean
#print(np.shape(img_meansub))
#viewer.add_image(img_meansub, name='meansub', colormap='inferno', visible=True)

# diff img - time
#img_diff_t = np.diff(imgstack_sm,axis=0)
#print(np.shape(img_diff_t))
#viewer.add_image(img_diff_t, name='diff-t', colormap='inferno', visible=True)

# diff img - x
img_diff_x = np.diff(imgstack_sm,axis=2)
viewer.add_image(img_diff_x, name='diff-x', colormap='inferno', visible=True)

# diff img - t on x
#img_diff_t_x = np.diff(img_diff_x,axis=0)
#print(np.shape(img_diff_t_x))
#viewer.add_image(img_diff_t_x, name='diff-t-x', colormap='inferno', visible=True)

# det coords on diff_x
coords_all = pd.DataFrame(columns=['id','t','x','y'])
for i in range(len(img_diff_x)):
    coords = peak_det(img_diff_x[i], testmode=False, min_dist=3, thresh_abs=40, num_peaks=500, noise_level=25, smoothing_radius=3)
    coords = np.flip(coords,axis=1)
    coords = coords[coords[:, 0].argsort()]
    coords_pd = pd.DataFrame(np.hstack((np.array(range(len(coords))).reshape(-1,1),i*np.ones(len(coords)).reshape(-1,1),coords)),columns=['id','t','x','y'])
    coords_all = coords_all.append(coords_pd)
    
thresh_dist = 5
yes = True
if yes:
    for i in range(1,int(max(coords_all['t']))+1):
        conn = list()
        coords_t1 = coords_all.loc[coords_all['t']==i]
        coords_t0 = coords_all.loc[coords_all['t']==i-1]
        #print('')
        #print(i)
        #print(coords_t1)
        #print(coords_t0)
        for n in range(len(coords_t1)):
            dists = list()
            for m in range(len(coords_t0)):
                p1 = (coords_t1['x'][n],coords_t1['y'][n])
                p0 = (coords_t0['x'][m],coords_t0['y'][m])
                dists.append(dist(p0,p1))
            #conn.append(np.argmin(dists))
            #print('')
            #print(coords_all.loc[(coords_all['t']==i) & (coords_all['x']==coords_t1.iloc[n]['x']) & (coords_all['y']==coords_t1.iloc[n]['y'])])
            if np.min(dists) < thresh_dist:
                coords_all.loc[(coords_all['t']==i) & (coords_all['x']==coords_t1.iloc[n]['x']) & (coords_all['y']==coords_t1.iloc[n]['y']),'id'] = int(coords_t0.iloc[np.argmin(dists)]['id'])
            else:
                coords_all.loc[(coords_all['t']==i) & (coords_all['x']==coords_t1.iloc[n]['x']) & (coords_all['y']==coords_t1.iloc[n]['y']),'id'] = int(np.max(coords_all['id'])+1)
            #print(coords_all.loc[(coords_all['t']==i) & (coords_all['x']==coords_t1.iloc[n]['x']) & (coords_all['y']==coords_t1.iloc[n]['y'])])
            
viewer.add_tracks(coords_all)

C:\Users\Jonatan\miniconda3\envs\smartstedanalysis\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)
